In [18]:

#PREPROCESAMIENTO DE IMAGEN

from PIL import Image

def preprocesar_imagen_a_txt(ruta_archivo_imagen, ruta_archivo_salida="imagen_preprocesada.txt"):

    try:
        imagen_original = Image.open(ruta_archivo_imagen).convert("L").resize((800, 600))
        lista_pixeles = list(imagen_original.getdata())
        pixeles_normalizados = [valor_pixel / 255.0 for valor_pixel in lista_pixeles]

        with open(ruta_archivo_salida, "w") as archivo_salida:
            ancho, alto = imagen_original.size 
            for i in range(alto):
                inicio_fila = i * ancho
                fin_fila = (i + 1) * ancho
                fila_pixeles = pixeles_normalizados[inicio_fila:fin_fila]
            
                archivo_salida.write(" ".join(map(str, fila_pixeles)) + "\n")
        print(f"Imagen '{ruta_archivo_imagen}' preprocesada y guardada en '{ruta_archivo_salida}'.")

    except FileNotFoundError:
        print(f"Error: No se encontró el archivo de imagen en '{ruta_archivo_imagen}'.")
    except Exception as e:
        print(f"Ocurrió un error inesperado durante el preprocesamiento: {e}")

preprocesar_imagen_a_txt("yo.jpeg", "imagen_si_procesada.txt")
preprocesar_imagen_a_txt("fotografia2.png", "imagen_no1_procesada.txt")
preprocesar_imagen_a_txt("fotografia3.jpg", "imagen_no2_procesada.txt")
preprocesar_imagen_a_txt("fotografia4.jpg", "imagen_no3_procesada.txt")


Imagen 'yo.jpeg' preprocesada y guardada en 'imagen_si_procesada.txt'.
Imagen 'fotografia2.png' preprocesada y guardada en 'imagen_no1_procesada.txt'.
Imagen 'fotografia3.jpg' preprocesada y guardada en 'imagen_no2_procesada.txt'.
Imagen 'fotografia4.jpg' preprocesada y guardada en 'imagen_no3_procesada.txt'.


In [20]:

#Componentes de la red neuronal

import math 
import random 

def cargar_pixeles_desde_txt(ruta_archivo_txt):
    try:
        with open(ruta_archivo_txt) as archivo:
            datos_pixeles = [float(pixel) for linea in archivo for pixel in linea.strip().split()]
        return datos_pixeles
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo de datos en '{ruta_archivo_txt}'.")
        return [] 
    except Exception as e:
        print(f"Ocurrió un error inesperado al cargar los datos: {e}")
        return []


# Funciones de activacion sigmoide y su derivada

def funcion_sigmoide(x):
    if x < -700: 
        return 0.0
    return 1 / (1 + math.exp(-x))

def derivada_funcion_sigmoide(salida_sigmoide):
    return salida_sigmoide * (1 - salida_sigmoide)


#Clase de la red neuronal

class RedNeuronalSimple:
    def __init__(self, num_neuronas_entrada, num_neuronas_ocultas, num_neuronas_salida):

        self.num_neuronas_entrada = num_neuronas_entrada
        self.num_neuronas_ocultas = num_neuronas_ocultas
        self.num_neuronas_salida = num_neuronas_salida

        self.pesos_capa_entrada_a_oculta = [
            [random.uniform(-0.5, 0.5) for _ in range(self.num_neuronas_entrada)]
            for _ in range(self.num_neuronas_ocultas)
        ]
# Biases para la capa oculta
        self.bias_capa_oculta = [random.uniform(-0.5, 0.5) for _ in range(self.num_neuronas_ocultas)]

# Pesos entre capa oculta y capa de salida
        self.pesos_capa_oculta_a_salida = [
            [random.uniform(-0.5, 0.5) for _ in range(self.num_neuronas_ocultas)]
            for _ in range(self.num_neuronas_salida)
        ]
# Biases para la capa de salida
        self.bias_capa_salida = [random.uniform(-0.5, 0.5) for _ in range(self.num_neuronas_salida)]

# Variables para almacenar activaciones intermedias durante forward pass
        self.activaciones_capa_oculta = [0.0] * self.num_neuronas_ocultas
        self.salida_red = [0.0] * self.num_neuronas_salida

    def propagacion_hacia_adelante(self, datos_entrada):
        if len(datos_entrada) != self.num_neuronas_entrada:
             raise ValueError(f"Tamaño de entrada incorrecto. Se esperaban {self.num_neuronas_entrada} valores, pero se recibieron {len(datos_entrada)}")

# Calcular activaciones de la capa oculta
        for i in range(self.num_neuronas_ocultas):
            suma_ponderada_oculta = sum(
                peso * valor_entrada for peso, valor_entrada in zip(self.pesos_capa_entrada_a_oculta[i], datos_entrada)
            ) + self.bias_capa_oculta[i]
            self.activaciones_capa_oculta[i] = funcion_sigmoide(suma_ponderada_oculta)

# Calcular activaciones de la capa de salida
        for i in range(self.num_neuronas_salida):
            suma_ponderada_salida = sum(
                peso * activacion_oculta for peso, activacion_oculta in zip(self.pesos_capa_oculta_a_salida[i], self.activaciones_capa_oculta)
            ) + self.bias_capa_salida[i]
            self.salida_red[i] = funcion_sigmoide(suma_ponderada_salida)

        return self.salida_red

    def retropropagacion(self, datos_entrada, salida_objetivo, tasa_aprendizaje=0.01):
        if len(salida_objetivo) != self.num_neuronas_salida:
             raise ValueError(f"Tamaño de salida objetivo incorrecto. Se esperaban {self.num_neuronas_salida} valores, pero se recibieron {len(salida_objetivo)}")

#Calcular el error en la capa de salida
        errores_capa_salida = [
            (salida_objetivo[i] - self.salida_red[i]) * derivada_funcion_sigmoide(self.salida_red[i])
            for i in range(self.num_neuronas_salida)
        ]

#Calcular el error en la capa oculta (propagado desde la salida)
        errores_capa_oculta = [0.0] * self.num_neuronas_ocultas
        for i in range(self.num_neuronas_ocultas):
            suma_error_ponderado = sum(
                self.pesos_capa_oculta_a_salida[j][i] * errores_capa_salida[j]
                for j in range(self.num_neuronas_salida)
            )
            errores_capa_oculta[i] = suma_error_ponderado * derivada_funcion_sigmoide(self.activaciones_capa_oculta[i])

#Actualizar pesos y biases de la capa de salida
        for i in range(self.num_neuronas_salida):
            for j in range(self.num_neuronas_ocultas):
                ajuste_peso = tasa_aprendizaje * errores_capa_salida[i] * self.activaciones_capa_oculta[j]
                self.pesos_capa_oculta_a_salida[i][j] += ajuste_peso
            ajuste_bias = tasa_aprendizaje * errores_capa_salida[i]
            self.bias_capa_salida[i] += ajuste_bias

#Actualizar pesos y biases de la capa oculta
        for i in range(self.num_neuronas_ocultas):
            for j in range(self.num_neuronas_entrada):
                ajuste_peso = tasa_aprendizaje * errores_capa_oculta[i] * datos_entrada[j]
                self.pesos_capa_entrada_a_oculta[i][j] += ajuste_peso
            ajuste_bias = tasa_aprendizaje * errores_capa_oculta[i]
            self.bias_capa_oculta[i] += ajuste_bias


In [22]:

#Entrenamiento de la red nueronal

datos_entrenamiento = [
    ("imagen_si_procesada.txt", [1.0]),     
    ("imagen_no1_procesada.txt", [0.0]),    
    ("imagen_no2_procesada.txt", [0.0]),
    ("imagen_no3_procesada.txt", [0.0])     
                      ]
dimension_entrada = 800 * 600 
neuronas_ocultas = 100         
neuronas_salida = 1            

red_neuronal = RedNeuronalSimple(dimension_entrada, neuronas_ocultas, neuronas_salida)

numero_epocas = 50 
tasa_ap = 0.05 

print("\n--- INICIANDO ENTRENAMIENTO ---")
for epoca in range(numero_epocas):
    error_total_epoca = 0
    
    for ruta_archivo_entrenamiento, etiqueta_esperada in datos_entrenamiento:  
        datos_imagen = cargar_pixeles_desde_txt(ruta_archivo_entrenamiento)

        if datos_imagen:
            salida_predicha = red_neuronal.propagacion_hacia_adelante(datos_imagen)
            
            error_ejemplo = sum([(etiqueta_esperada[i] - salida_predicha[i])**2 for i in range(neuronas_salida)]) / neuronas_salida
            error_total_epoca += error_ejemplo

            red_neuronal.retropropagacion(datos_imagen, etiqueta_esperada, tasa_aprendizaje=tasa_ap)

    if (epoca + 1) % 10 == 0:
        error_promedio_epoca = error_total_epoca / len(datos_entrenamiento)
        print(f"Época {epoca + 1}/{numero_epocas} completada. Error promedio: {error_promedio_epoca:.6f}")

print("--- ENTRENAMIENTO COMPLETADO ---")


--- INICIANDO ENTRENAMIENTO ---
Época 10/50 completada. Error promedio: 0.153401
Época 20/50 completada. Error promedio: 0.090265
Época 30/50 completada. Error promedio: 0.050310
Época 40/50 completada. Error promedio: 0.034852
Época 50/50 completada. Error promedio: 0.021229
--- ENTRENAMIENTO COMPLETADO ---


In [24]:

#Prueba del modelo ya entrenado

print("\n--- PROBANDO EL MODELO ---")

ruta_imagen_prueba_txt = "imagen_si_procesada.txt"
datos_imagen_prueba = cargar_pixeles_desde_txt(ruta_imagen_prueba_txt)

if datos_imagen_prueba:
    prediccion_red = red_neuronal.propagacion_hacia_adelante(datos_imagen_prueba)
    salida_bruta = prediccion_red[0] 
    umbral_clasificacion = 0.5
    clasificacion = "SI es la imagen" if salida_bruta >= umbral_clasificacion else "NO es la imagen"

    print(f"\nResultado para la imagen de prueba ('{ruta_imagen_prueba_txt}'):")
    print(f"Salida bruta de la red: {salida_bruta:.4f}")
    print(f"Clasificación (umbral {umbral_clasificacion}): {clasificacion}")
else:
    print(f"No se pudo cargar la imagen de prueba '{ruta_imagen_prueba_txt}' para realizar la predicción.")

print("\n--- FIN DEL SCRIPT ---")


--- PROBANDO EL MODELO ---

Resultado para la imagen de prueba ('imagen_si_procesada.txt'):
Salida bruta de la red: 0.7915
Clasificación (umbral 0.5): SI es la imagen

--- FIN DEL SCRIPT ---


In [26]:

#Prueba del modelo ya entrenado

print("\n--- PROBANDO EL MODELO ---")

ruta_imagen_prueba_txt = "imagen_no1_procesada.txt"
datos_imagen_prueba = cargar_pixeles_desde_txt(ruta_imagen_prueba_txt)

if datos_imagen_prueba:
    prediccion_red = red_neuronal.propagacion_hacia_adelante(datos_imagen_prueba)
    salida_bruta = prediccion_red[0] 
    umbral_clasificacion = 0.5
    clasificacion = "SI es la imagen" if salida_bruta >= umbral_clasificacion else "NO es la imagen"

    print(f"\nResultado para la imagen de prueba ('{ruta_imagen_prueba_txt}'):")
    print(f"Salida bruta de la red: {salida_bruta:.4f}")
    print(f"Clasificación (umbral {umbral_clasificacion}): {clasificacion}")
else:
    print(f"No se pudo cargar la imagen de prueba '{ruta_imagen_prueba_txt}' para realizar la predicción.")

print("\n--- FIN DEL SCRIPT ---")


--- PROBANDO EL MODELO ---

Resultado para la imagen de prueba ('imagen_no1_procesada.txt'):
Salida bruta de la red: 0.0674
Clasificación (umbral 0.5): NO es la imagen

--- FIN DEL SCRIPT ---


In [28]:

#Prueba del modelo ya entrenado

print("\n--- PROBANDO EL MODELO ---")

ruta_imagen_prueba_txt = "imagen_no2_procesada.txt"
datos_imagen_prueba = cargar_pixeles_desde_txt(ruta_imagen_prueba_txt)

if datos_imagen_prueba:
    prediccion_red = red_neuronal.propagacion_hacia_adelante(datos_imagen_prueba)
    salida_bruta = prediccion_red[0] 
    umbral_clasificacion = 0.5
    clasificacion = "SI es la imagen" if salida_bruta >= umbral_clasificacion else "NO es la imagen"

    print(f"\nResultado para la imagen de prueba ('{ruta_imagen_prueba_txt}'):")
    print(f"Salida bruta de la red: {salida_bruta:.4f}")
    print(f"Clasificación (umbral {umbral_clasificacion}): {clasificacion}")
else:
    print(f"No se pudo cargar la imagen de prueba '{ruta_imagen_prueba_txt}' para realizar la predicción.")

print("\n--- FIN DEL SCRIPT ---")


--- PROBANDO EL MODELO ---

Resultado para la imagen de prueba ('imagen_no2_procesada.txt'):
Salida bruta de la red: 0.1641
Clasificación (umbral 0.5): NO es la imagen

--- FIN DEL SCRIPT ---


In [30]:

#Prueba del modelo ya entrenado

print("\n--- PROBANDO EL MODELO ---")

ruta_imagen_prueba_txt = "imagen_no3_procesada.txt"
datos_imagen_prueba = cargar_pixeles_desde_txt(ruta_imagen_prueba_txt)

if datos_imagen_prueba:
    prediccion_red = red_neuronal.propagacion_hacia_adelante(datos_imagen_prueba)
    salida_bruta = prediccion_red[0] 
    umbral_clasificacion = 0.5
    clasificacion = "SI es la imagen" if salida_bruta >= umbral_clasificacion else "NO es la imagen"

    print(f"\nResultado para la imagen de prueba ('{ruta_imagen_prueba_txt}'):")
    print(f"Salida bruta de la red: {salida_bruta:.4f}")
    print(f"Clasificación (umbral {umbral_clasificacion}): {clasificacion}")
else:
    print(f"No se pudo cargar la imagen de prueba '{ruta_imagen_prueba_txt}' para realizar la predicción.")

print("\n--- FIN DEL SCRIPT ---")


--- PROBANDO EL MODELO ---

Resultado para la imagen de prueba ('imagen_no3_procesada.txt'):
Salida bruta de la red: 0.0673
Clasificación (umbral 0.5): NO es la imagen

--- FIN DEL SCRIPT ---
